In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logreg').getOrCreate()

In [4]:
data = spark.read.csv("D:\\Udemy course\\Spark\\MLLib\\titanic.csv", inferSchema=True, header=True)
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [5]:
data.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [17]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [19]:
#encoding categorical features.....Sex and Embarked

from pyspark.ml.feature import StringIndexer, OneHotEncoder

sex_index = StringIndexer(inputCol='Sex', outputCol='Sex_index')
sex_vec = OneHotEncoder(inputCol='Sex_index', outputCol='sex_vec')

embark_index = StringIndexer(inputCol='Embarked', outputCol='embark_index')
embark_vec = OneHotEncoder(inputCol='embark_index', outputCol='embark_vec')

In [21]:
#setting up the features into vector...
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = ['Pclass','sex_vec','Age','SibSp','Parch','Fare','embark_vec'],
                          outputCol = 'features')

In [22]:
#loading model
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol='features',labelCol='Survived')

In [25]:
#creating a pipeline to set up our work
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[sex_index, embark_index, sex_vec, embark_vec, assembler, lr])

In [35]:
#selecting our data....
final_data = data.select(['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','Survived'])
my_final = final_data.na.drop()

#split data into train and test....
train,test = my_final.randomSplit([0.8,0.2])

In [36]:
#fit the training data to the pipeline....
fit_model = pipeline.fit(train)

In [37]:
#transform and predict on the test data.....
results = fit_model.transform(test)

In [38]:
#Evaluate the test results.....
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluation = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [39]:
results.select(['Survived','prediction']).show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
+--------+----------+
only showing top 20 rows



In [41]:
auc = evaluation.evaluate(results)
print("area under curve score: {0}".format(auc))

area under curve score: 0.8421516754850088
